# Frame the problem

1. __What is the objective?__ To understand/quantify the causal relationship between air quality and health outcomes. It is widely known that degredations in air quality can worsen conditions such as asthma and heart disease. For point-in-time events (e.g. forest fires, storms, etc.) that lead to rapid decreases in air quality, are we able to quantify the causal impact on health outcomes? For example, how many incremental emergency room visits for asthma attacks and heart attacks can be causally linked to the LA fires of early 2025? 

2. __How will the solution be used?__ This will just be a data analysis for now. In theory, it is possible for 

3. __What are current solutions?__

4. __How should the problem be framed?__

5. __How should performance be measured?__

6. __What is the minimum performance needed?__

7. __What are comparable problems?__

8. __Have any assumptions been made? Verify them if possible.__